In [1]:
import numpy as np
import gvar as gv
import sys

#import warnings  
#warnings.simplefilter(action="default")

sys.path.append('../')

from h5glance import H5Glance 
import os
import lsqfit

In [2]:
os.chdir('../../hyperon-xpt-master/xpt')

In [3]:
import fit_analysis as fa 
import fit_routine as fit
import i_o 

In [4]:
from importlib import reload
reload(i_o)
reload(fa)
reload(fit)

<module 'fit_routine' from '/Users/grantdb/hyperon-xpt-master/xpt/fit_routine.py'>

In [5]:
file = '../data/hyperon_data.h5'
#H5Glance(file)

In [6]:
file2 = '../data/input_data.h5'
#H5Glance(file2)

In [7]:
input_output = i_o.InputOutput()
data,ensembles = input_output.get_data(scheme='w0_imp')

for j,ens in enumerate(ensembles):
    print(ens, data['m_xi'][j])

/Users/grantdb/hyperon-xpt-master/xpt/i_o.py:101: UserWarning: sample sizes differ for different entries: 5000 5001
  gv_data[ens] = gv.dataset.avg_data(gv_data[ens], bstrap=True)
a09m135 1282(12)
a09m220 1318(11)
a09m310 1370(11)
a09m350 1388(11)
a09m400 1411(12)
a12m130 1271.5(9.6)
a12m220L 1283(10)
a12m310 1318.6(9.9)
a12m350 1336(10)
a12m400 1358(10)
a15m135XL 1258.0(8.6)
a15m220 1257.6(8.5)
a15m310 1294.3(8.7)
a15m350 1302.1(9.7)
a15m400 1327.5(9.8)


In [8]:
prior = {}

# not-even leading order 
prior['m_{xi,0}'] = gv.gvar(1000, 1000) # MeV
prior['m_{xi_st,0}'] = gv.gvar(1100, 1000) # MeV
prior['m_{lam,0}'] = gv.gvar(1200, 1000) 
prior['m_{sigma,0}'] = gv.gvar(1300, 1000) 
prior['m_{sigma_st,0}'] = gv.gvar(1400, 1000)


# lo
prior['s_{xi}'] = gv.gvar(0, 5)
prior['s_{xi,bar}'] = gv.gvar(0, 5)
prior['s_{lam}'] = gv.gvar(0, 5)
prior['s_{sigma,bar}'] = gv.gvar(0, 5)
prior['s_{sigma}'] = gv.gvar(0, 5)


# nlo
prior['g_{xi,xi}'] = gv.gvar(0.3, 3)
prior['g_{xi_st,xi}'] = gv.gvar(0.7, 3)
prior['g_{xi_st,xi_st}'] = gv.gvar(-.5, 3)
prior['g_{lam,sigma}'] = gv.gvar(0, 5)
prior['g_{lam,sigma_st}'] =gv.gvar(0, 5)
prior['g_{sigma,sigma}'] = gv.gvar(0, 5)
prior['g_{sigma_st,sigma}'] = gv.gvar(0, 5)
prior['g_{sigma_st,sigma_st}']= gv.gvar(0, 5)


# n2lo
prior['a_{xi,4}'] = gv.gvar(0, 5)
prior['a_{xi_st,4}'] = gv.gvar(0, 5)
prior['b_{xi,4}'] = gv.gvar(0, 5)
prior['b_{xi_st,4}'] = gv.gvar(0, 5)
prior['a_{sigma,4}'] = gv.gvar(0, 5)
prior['a_{sigma_st,4}'] = gv.gvar(0, 5)
prior['b_{sigma,4}'] = gv.gvar(0, 5)
prior['b_{sigma_st,4}'] = gv.gvar(0, 5)
prior['a_{lam,4}'] = gv.gvar(0, 5)
prior['b_{lam,4}'] = gv.gvar(0, 5)
# note: no lo terms for taylor 

# latt/strange nlo
prior['d_{xi,a}'] = gv.gvar(0, 1)
prior['d_{xi_st,a}'] = gv.gvar(0, 1)
prior['d_{xi,s}'] = gv.gvar(0, 1)
prior['d_{xi_st,s}'] = gv.gvar(0, 1)  
prior['d_{lam,s}'] = gv.gvar(0, 1)
prior['d_{lam,a}'] = gv.gvar(0, 1)
prior['d_{sigma_st,a}'] = gv.gvar(0, 1) 
prior['d_{sigma_st,s}'] = gv.gvar(0, 1)
prior['d_{sigma,s}'] = gv.gvar(0, 1) 
prior['d_{sigma,a}'] = gv.gvar(0, 1)



# latt n2lo
prior['d_{xi,aa}'] = gv.gvar(0, 1)
prior['d_{xi,al}'] = gv.gvar(0, 1)
prior['d_{xi,as}'] = gv.gvar(0, 1)
prior['d_{xi,ls}'] = gv.gvar(0, 1)
prior['d_{xi,ss}'] = gv.gvar(0, 1)

prior['d_{xi_st,aa}'] = gv.gvar(0, 1)
prior['d_{xi_st,al}'] = gv.gvar(0, 1) 
prior['d_{xi_st,as}'] = gv.gvar(0, 1)
prior['d_{xi_st,ls}'] = gv.gvar(0, 1) 
prior['d_{xi_st,ss}'] = gv.gvar(0, 1)

prior['d_{lam,aa}'] = gv.gvar(0, 1)
prior['d_{lam,al}'] = gv.gvar(0, 1)
prior['d_{lam,as}'] = gv.gvar(0, 1)
prior['d_{lam,ls}'] = gv.gvar(0, 1)
prior['d_{lam,ss}'] = gv.gvar(0, 1)

prior['d_{sigma,aa}'] = gv.gvar(0, 1)
prior['d_{sigma,al}'] = gv.gvar(0, 1)
prior['d_{sigma,as}'] = gv.gvar(0, 1)
prior['d_{sigma,ls}'] = gv.gvar(0, 1)
prior['d_{sigma,ss}'] = gv.gvar(0, 1)

prior['d_{sigma_st,aa}'] = gv.gvar(0, 1)
prior['d_{sigma_st,al}'] = gv.gvar(0, 1) 
prior['d_{sigma_st,as}'] = gv.gvar(0, 1)
prior['d_{sigma_st,ls}'] = gv.gvar(0, 1) 
prior['d_{sigma_st,ss}'] = gv.gvar(0, 1)

In [9]:
model_info = {}
model_info['particles'] = ['xi', 'xi_st','sigma', 'sigma_st', 'lam']
model_info['eps2a_defn'] = 'w0_imp'
#model_info['name'] = 'test'
model_info['order_chiral']  = 'n2lo'
model_info['order_disc'] = 'n2lo'
model_info['order_strange'] = 'n2lo'
model_info['include_log'] = True
model_info['xpt'] = True
#model_info['full_formulae'] = True

In [10]:
def _make_prior(data,prior):
    
    new_prior = {}
    for key in prior:
        new_prior[key] = prior[key]
    for key in ['m_pi', 'm_k', 'lam_chi', 'a/w', 'm_delta']:
        new_prior[key] = data[key]
    return new_prior

In [11]:

data,ensembles = input_output.get_data(scheme='w0_imp')
prior = _make_prior(data,prior)
phys_point_data = input_output.get_phys_point_data()

In [12]:
prior['m_delta']

array([1382(17), 1346(66), 1464(69), 1497(19), 1556(14), 1370(21),
       1402(11), 1441(30), 1459(23), 1498(15), 1450(19), 1404(14),
       1439.2(9.9), 1414(34), 1488(13)], dtype=object)

In [13]:
print(phys_point_data)

{'a/w': 0(0), 'a': 0(0), 'alpha_s': 0(0), 'L': inf(0), 'hbarc': 197.327(0), 'lam_chi': 1157.0(7.2), 'm_pi': 134.80(30), 'm_k': 494.20(30), 'm_xi': 1318.28(11), 'm_xi_st': 1533.40(34), 'm_lam': 1118.697(35), 'm_sigma': 1198.82(30), 'm_sigma_st': 1388.35(58)}


In [14]:
sf_phys = ((2*phys_point_data['m_k']**2- phys_point_data['m_pi']**2) / phys_point_data['lam_chi']**2)
print(sf_phys)

0.3513(44)


In [15]:
#run this until it works... 
fit_test = fit.fit_routine(prior=prior, data=data, model_info=model_info)

In [16]:
print(fit_test.fit)


Least Square Fit:
  chi2/dof [dof] = 0.87 [75]    Q = 0.78    logGBF = -347.05

Parameters:
             m_{xi,0}          1337 (16)        [  1000 (1000) ]  
          m_{xi_st,0}          1574 (22)        [  1100 (1000) ]  
            m_{lam,0}          1100 (17)        [  1200 (1000) ]  
          m_{sigma,0}          1229 (15)        [  1300 (1000) ]  
       m_{sigma_st,0}          1510 (19)        [  1400 (1000) ]  
               s_{xi}          0.90 (74)        [    0.0 (5.0) ]  
           s_{xi,bar}        0.02 (5.00)        [    0.0 (5.0) ]  
              s_{lam}          2.29 (74)        [    0.0 (5.0) ]  
        s_{sigma,bar}          0.3 (1.9)        [    0.0 (5.0) ]  
            s_{sigma}          0.74 (92)        [    0.0 (5.0) ]  
            g_{xi,xi}         -0.62 (72)        [    0.3 (3.0) ]  
         g_{xi_st,xi}          0.1 (2.8)        [    0.7 (3.0) ]  
      g_{xi_st,xi_st}       -0.08 (2.79)        [   -0.5 (3.0) ]  
        g_{lam,sigma}        0.05 (4.

In [17]:
post = {}
my_fit = fit_test.fit
post = my_fit.p
for key in post.keys():
    print(key)
#print(post.keys())

m_{xi,0}
m_{xi_st,0}
m_{lam,0}
m_{sigma,0}
m_{sigma_st,0}
s_{xi}
s_{xi,bar}
s_{lam}
s_{sigma,bar}
s_{sigma}
g_{xi,xi}
g_{xi_st,xi}
g_{xi_st,xi_st}
g_{lam,sigma}
g_{lam,sigma_st}
g_{sigma,sigma}
g_{sigma_st,sigma}
g_{sigma_st,sigma_st}
a_{xi,4}
a_{xi_st,4}
b_{xi,4}
b_{xi_st,4}
a_{sigma,4}
a_{sigma_st,4}
b_{sigma,4}
b_{sigma_st,4}
a_{lam,4}
b_{lam,4}
d_{xi,a}
d_{xi_st,a}
d_{xi,s}
d_{xi_st,s}
d_{lam,s}
d_{lam,a}
d_{sigma_st,a}
d_{sigma_st,s}
d_{sigma,s}
d_{sigma,a}
d_{xi,aa}
d_{xi,al}
d_{xi,as}
d_{xi,ls}
d_{xi,ss}
d_{xi_st,aa}
d_{xi_st,al}
d_{xi_st,as}
d_{xi_st,ls}
d_{xi_st,ss}
d_{lam,aa}
d_{lam,al}
d_{lam,as}
d_{lam,ls}
d_{lam,ss}
d_{sigma,aa}
d_{sigma,al}
d_{sigma,as}
d_{sigma,ls}
d_{sigma,ss}
d_{sigma_st,aa}
d_{sigma_st,al}
d_{sigma_st,as}
d_{sigma_st,ls}
d_{sigma_st,ss}
m_pi
m_k
lam_chi
a/w
m_delta


In [18]:
my_fit = fit_test.fit
post = my_fit.p
fit_test = fit.fit_routine(prior=post, data=phys_point_data, model_info=model_info)


In [19]:
fit_analysis = fa.fit_analysis(
    phys_point_data=phys_point_data, 
    data=data, 
    model_info=model_info,
    prior=prior)

AttributeError: can't set attribute

In [21]:
print(fa.fit_analysis.fitfcn(p=post, data=phys_point_data))

TypeError: fitfcn() missing 1 required positional argument: 'self'